# オバマのスピーチから翻訳吹き替え動画を作ろう


動作環境について

Webブラウザについて

safariだと動作しないことが確認されています. 

Chromeでは動作することが確認できています.

## ESPnet2(ASR, 音声->テキスト)

EspNet2によるAutomatic Sound Recognition

In [ ]:
# NOTE: pip shows imcompatible errors due to preinstalled libraries but you do not need to care
!pip install -q espnet==0.10.0
!pip install -q espnet_model_zoo
!pip install googletrans==4.0.0-rc1

In [2]:
#@title Choose English ASR model { run: "auto" }

lang = 'en'
fs = 16000 #@param {type:"integer"}
tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [ ]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

Recognize your pre-recordings

今はサンプリング周波数16000Hzのwavファイルを受け付けるようになっています。使うファイルによって適宜コードを変えてください。

音声ファイルを用意するのが面倒くさい場合は以下のセルでダウンロードできるwavファイルを利用してください。

In [ ]:
!pip install gdown
!gdown --id "1PgfAxfliwmgiDX4dJgHXyTsMmpDbuHtM"
#https://drive.google.com/file/d/1PgfAxfliwmgiDX4dJgHXyTsMmpDbuHtM/view?usp=sharing

In [ ]:
from google.colab import files
from IPython.display import display, Audio
import soundfile
import librosa.display
import matplotlib.pyplot as plt

print("upload your wav file:")
uploaded = files.upload()

for file_name in uploaded.keys():
  speech, rate = soundfile.read(file_name)
  assert rate == fs, "mismatch in sampling rate"
  nbests = speech2text(speech)
  text, *_ = nbests[0]
  text = text_normalizer(text)

  print(f"Input Speech: {file_name}")
  display(Audio(speech, rate=rate))
  librosa.display.waveplot(speech, sr=rate)
  plt.show()
  print(f"ASR hypothesis: {text}")
  print("*" * 50)

## 翻訳(英語テキスト->日本語テキスト)

google翻訳による翻訳

In [ ]:
from googletrans import Translator
# pip install googletrans==4.0.0-rc1

translator = Translator()
result = translator.translate(text, dest='ja', src='en')
data = result.text
print(data)

DeepL APIを持っている場合は以下を使ってください

DeepL APIの認証キーが必要です

In [ ]:
import requests

print("upload your DeepL API key:")
params = {"auth_key": input(),
         "text": text,
         "source_lang": 'EN',
         "target_lang": 'JA'
         }

request = requests.post("https://api-free.deepl.com/v2/translate", data=params) # free用のURL、有料版はURLが異なります
result = request.json()
data = result["translations"][0]["text"]
print(data)

## Google APIによる音声合成

In [ ]:
! mkdir data
! mkdir data/testset_api

Google API tokenをアップロード

持っていない場合は以下のセルでダウンロードできる音声を利用して、それ以降の音声合成の部分は飛ばしてください。

In [ ]:
!pip install gdown
%cd /content/data/testset_api/
!gdown --id "1oX1RnhgUQ94v1VL2MmZX4tB1d70kIcNv"
#https://drive.google.com/file/d/1oX1RnhgUQ94v1VL2MmZX4tB1d70kIcNv/view?usp=sharing

In [ ]:
from google.colab import files

# google access tokenをアップロードしてください
print("upload your google access token:")
uploaded = files.upload()
# 自身の環境で
# $ gcloud auth print-access-token
# として出てきたものをtokenとして入力してください
print("upload the result of $ gcloud auth print-access-token on your terminal:")
token = input()

In [ ]:
import os
import base64
import numpy as np

import urllib.request
import json
import subprocess as sp

def makeRequestDict(txt: str):
    # """
    # Google Text-To-Speechへリクエストのための情報を生成する
    # SSMLには未対応

    # Args:
    #     txt(in): 音声合成するテキスト

    # Returns:
    #     音声合成するために必要な情報をdictで返却する
    # """
    dat = {"audioConfig": {
        "audioEncoding": "LINEAR16",
        "pitch": -8,
        "speakingRate": 1,
        "sampleRateHertz": 16000
      },
      "voice": {
        "languageCode": "ja-JP",
        "name": "ja-JP-Wavenet-D"
      }
    }

    dat["input"] = {"text": txt}
    return dat

def output_wav(dat: dict, ofile: str):
    # """
    # Google Text-To-Speechへリクエストした結果を元に音声データにしてファイルに書き込む

    # Args:
    #     dat(in):   リクエストした結果得られたJSON文字列をdictにしたもの
    #     ofile(in): 音声データを書き出すファイル名
    # """
    b64str = dat["audioContent"]
    binary = base64.b64decode(b64str)
    dat = np.frombuffer(binary,dtype=np.uint8)
    with open(ofile,"wb") as f:
        f.write(dat)

def gtts(txt: str, ofile: str):

    dat = makeRequestDict(txt)
    req_data = json.dumps(dat).encode()

    url = 'https://texttospeech.googleapis.com/v1beta1/text:synthesize'
    req_header = {
            'Authorization': f"Bearer {token}",
            'Content-Type': 'application/json; charset=utf-8',
    }
    req = urllib.request.Request(url, data=req_data, method='POST', headers=req_header)

    try:
        with urllib.request.urlopen(req) as response:
            dat = response.read()
            body = json.loads(dat)
            output_wav(body, ofile)
            print("done..")
    except urllib.error.URLError as e:
        print("error happen...")
        print(e.reason)
        print(e)

In [ ]:
output_wav_str = "data/testset_api/japaneseAPI.wav"
gtts(data, output_wav_str)

## CycleGAN-VC2(日本語API->オバマの日本語音声)


### 以下のモデルから好きな方を使ってください.

### epoch4000, データ数100, 加工なし, の条件でtrainしたモデル

**・epoch4000, データ数100, 加工なし**

**「ランタイム」から「ランタイプのタイプを変更」からハードウェアアクセサレータをGPUに設定してください。**

In [ ]:
#githubからコードをダウンロードし, googledriveから訓練済みモデルをダウンロードする
!mkdir /content/CycleGAN-VC2_e4000_d100_nyes
%cd /content/CycleGAN-VC2_e4000_d100_nyes
!git clone https://github.com/aiutarsi/CycleGAN-VC2.git
!mkdir /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/model_checkpoint
!pip install gdown
%cd /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/model_checkpoint
!gdown --id "1g0HH6ICByuRmPErZOG29WYTxyrXF9F1v"
#https://drive.google.com/file/d/1g0HH6ICByuRmPErZOG29WYTxyrXF9F1v/view?usp=sharing
#必要なライブラリをインストールする
%cd /content/
!pip install -r requirements.txt
!pip install librosa==0.5.1

今、`CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/data/test_ja`に生成された日本語のAPIの音声(16kHz, 1チャンネル, wavファイル)が入っています. この音声が以下のセルを実行すると, `CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/converted_sound/test_ja`にオバマの声に変換した日本語の音声のファイルが生成されます.

逆に,
`CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/data/test_obama`に英語のオバマの音声(16kHz, 1チャンネル, 4秒以上, wavファイル)を置いて、以下のセルを実行すると, `CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/converted_sound/test_obama`に日本語のAPIの声に変換したオバマの音声のファイルが生成されます.

In [11]:
#日本語のAPIの音声をCycleGAN-VC2側のフォルダに移動する
%cp /content/data/testset_api/japaneseAPI.wav /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/data/test_ja/japaneseAPI.wav

In [ ]:
#テストを実行する
!python3 /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/test.py

### epoch10000, データ数100, 加工なし, の条件でtrainしたモデル

**・epoch10000, データ数100, 加工なし**

**「ランタイム」から「ランタイプのタイプを変更」からハードウェアアクセサレータをGPUに設定してください。**

In [ ]:
#githubからコードをダウンロードし, googledriveから訓練済みモデルをダウンロードする
!mkdir /content/CycleGAN-VC2_e4000_d100_nyes
%cd /content/CycleGAN-VC2_e4000_d100_nyes
!git clone https://github.com/aiutarsi/CycleGAN-VC2.git
!mkdir /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/model_checkpoint
!pip install gdown
%cd /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/model_checkpoint
!gdown --id "1cTbRaQqtgNcDkd_vZ-rzkwk8dP05CVnJ"
#https://drive.google.com/file/d/1cTbRaQqtgNcDkd_vZ-rzkwk8dP05CVnJ/view?usp=sharing
#必要なライブラリをインストールする
%cd /content/
!pip install -r requirements.txt
!pip install librosa==0.5.1

今、CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/data/test_jaに生成された日本語のAPIの音声(16kHz, 1チャンネル, wavファイル)が入っています. この音声が以下のセルを実行すると, CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/converted_sound/test_jaにオバマの声に変換した日本語の音声のファイルが生成されます.

逆に, CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/data/test_obamaに英語のオバマの音声(16kHz, 1チャンネル, 4秒以上, wavファイル)を置いて、以下のセルを実行すると, CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/converted_sound/test_obamaに日本語のAPIの声に変換したオバマの音声のファイルが生成されます.

In [8]:
#日本語のAPIの音声をCycleGAN-VC2側のフォルダに移動する
%cp /content/data/testset_api/japaneseAPI.wav /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/data/test_ja/japaneseAPI.wav

In [ ]:
#テストを実行する
!python3 /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/test.py

## MakeItTalk(オバマの日本語音声+画像->動画)

MakeItTalkのgithubリポジトリのデモcolabのコードをほぼそのまま利用したものです.

準備

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
import subprocess
print(subprocess.getoutput('nvidia-smi'))

In [ ]:
print(subprocess.getoutput('ffmpeg'))

In [ ]:
!git clone https://github.com/yzhou359/MakeItTalk

In [ ]:
%cd /content/MakeItTalk
!export PYTHONPATH=/content/MakeItTalk:$PYTHONPATH
!pip install -r requirements.txt
!pip install tensorboardX

In [ ]:
!mkdir examples/dump
!mkdir examples/ckpt
!pip install gdown
!gdown -O examples/ckpt/ckpt_autovc.pth https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
!gdown -O examples/ckpt/ckpt_content_branch.pth https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
!gdown -O examples/ckpt/ckpt_speaker_branch.pth https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
!gdown -O examples/ckpt/ckpt_116_i2i_comb.pth https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
!gdown -O examples/dump/emb.pickle https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI

パッケージのインストール

In [38]:
import sys
sys.path.append("thirdparty/AdaptiveWingLoss")
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import time
import util.utils as util
from scipy.signal import savgol_filter
from src.approaches.train_audio2landmark import Audio2landmark_model

セットアップ

In [39]:
default_head_name = 'obama'           # the image name (with no .jpg) to animate
ADD_NAIVE_EYE = True                 # whether add naive eye blink
CLOSE_INPUT_FACE_MOUTH = False       # if your image has an opened mouth, put this as True, else False
AMP_LIP_SHAPE_X = 2.                 # amplify the lip motion in horizontal direction
AMP_LIP_SHAPE_Y = 2.                 # amplify the lip motion in vertical direction
AMP_HEAD_POSE_MOTION = 0.7           # amplify the head pose motion (usually smaller than 1.0, put it to 0. for a static head pose)

In [40]:
parser = argparse.ArgumentParser()
parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name))
parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')

parser.add_argument('--load_AUTOVC_name', type=str, default='examples/ckpt/ckpt_autovc.pth')
parser.add_argument('--load_a2l_G_name', type=str, default='examples/ckpt/ckpt_speaker_branch.pth')
parser.add_argument('--load_a2l_C_name', type=str, default='examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
parser.add_argument('--load_G_name', type=str, default='examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c

parser.add_argument('--amp_lip_x', type=float, default=AMP_LIP_SHAPE_X)
parser.add_argument('--amp_lip_y', type=float, default=AMP_LIP_SHAPE_Y)
parser.add_argument('--amp_pos', type=float, default=AMP_HEAD_POSE_MOTION)
parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
parser.add_argument('--add_audio_in', default=False, action='store_true')
parser.add_argument('--comb_fan_awing', default=False, action='store_true')
parser.add_argument('--output_folder', type=str, default='examples')

parser.add_argument('--test_end2end', default=True, action='store_true')
parser.add_argument('--dump_dir', type=str, default='', help='')
parser.add_argument('--pos_dim', default=7, type=int)
parser.add_argument('--use_prior_net', default=True, action='store_true')
parser.add_argument('--transformer_d_model', default=32, type=int)
parser.add_argument('--transformer_N', default=2, type=int)
parser.add_argument('--transformer_heads', default=2, type=int)
parser.add_argument('--spk_emb_enc_size', default=16, type=int)
parser.add_argument('--init_content_encoder', type=str, default='')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
parser.add_argument('--write', default=False, action='store_true')
parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
parser.add_argument('--emb_coef', default=3.0, type=float)
parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
parser.add_argument('--use_11spk_only', default=False, action='store_true')
parser.add_argument('-f')

opt_parser = parser.parse_args()

In [ ]:
#japaneseAPI.wav(オバマの日本語音声)ををMakeItTalkのディレクトリに移動する
%cd /content/MakeItTalk/examples
%cp /content/CycleGAN-VC2_e4000_d100_nyes/CycleGAN-VC2/converted_sound/test_ja/japaneseAPI.wav /content/MakeItTalk/examples/japaneseAPI.wav
%rm M6_04_16k.wav

イメージのロード+ランドマークの認識

In [ ]:
%cd /content/MakeItTalk/
img =cv2.imread('/content/MakeItTalk/examples/' + opt_parser.jpg)
predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, device='cpu', flip_input=True)
shapes = predictor.get_landmarks(img)
if (not shapes or len(shapes) != 1):
    print('Cannot detect face landmarks. Exit.')
    exit(-1)
shape_3d = shapes[0]

if(opt_parser.close_input_face_mouth):
    util.close_input_face_mouth(shape_3d)

オプション

In [43]:
shape_3d[48:, 0] = (shape_3d[48:, 0] - np.mean(shape_3d[48:, 0])) * 1.05 + np.mean(shape_3d[48:, 0]) # wider lips
shape_3d[49:54, 1] += 0.           # thinner upper lip
shape_3d[55:60, 1] -= 1.           # thinner lower lip
shape_3d[[37,38,43,44], 1] -=2.    # larger eyes
shape_3d[[40,41,46,47], 1] +=2.    # larger eyes

In [44]:
shape_3d, scale, shift = util.norm_input_face(shape_3d)

音声から推論

In [ ]:
au_data = []
au_emb = []
ains = glob.glob1('examples', '*.wav')
ains = [item for item in ains if item is not 'tmp.wav']
ains.sort()
for ain in ains:
    os.system('ffmpeg -y -loglevel error -i examples/{} -ar 16000 examples/tmp.wav'.format(ain))
    shutil.copyfile('examples/tmp.wav', 'examples/{}'.format(ain))

    # au embedding
    from thirdparty.resemblyer_util.speaker_emb import get_spk_emb
    me, ae = get_spk_emb('examples/{}'.format(ain))
    au_emb.append(me.reshape(-1))

    print('Processing audio file', ain)
    c = AutoVC_mel_Convertor('examples')

    au_data_i = c.convert_single_wav_to_autovc_input(audio_filename=os.path.join('examples', ain),
           autovc_model_path=opt_parser.load_AUTOVC_name)
    au_data += au_data_i
if(os.path.isfile('examples/tmp.wav')):
    os.remove('examples/tmp.wav')

# landmark fake placeholder
fl_data = []
rot_tran, rot_quat, anchor_t_shape = [], [], []
for au, info in au_data:
    au_length = au.shape[0]
    fl = np.zeros(shape=(au_length, 68 * 3))
    fl_data.append((fl, info))
    rot_tran.append(np.zeros(shape=(au_length, 3, 4)))
    rot_quat.append(np.zeros(shape=(au_length, 4)))
    anchor_t_shape.append(np.zeros(shape=(au_length, 68 * 3)))

if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_fl.pickle'))
if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))
if(os.path.exists(os.path.join('examples', 'dump', 'random_val_au.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_au.pickle'))
if (os.path.exists(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))

with open(os.path.join('examples', 'dump', 'random_val_fl.pickle'), 'wb') as fp:
    pickle.dump(fl_data, fp)
with open(os.path.join('examples', 'dump', 'random_val_au.pickle'), 'wb') as fp:
    pickle.dump(au_data, fp)
with open(os.path.join('examples', 'dump', 'random_val_gaze.pickle'), 'wb') as fp:
    gaze = {'rot_trans':rot_tran, 'rot_quat':rot_quat, 'anchor_t_shape':anchor_t_shape}
    pickle.dump(gaze, fp)

音声とランドマークの予測

In [ ]:
!pwd
model = Audio2landmark_model(opt_parser, jpg_shape=shape_3d)
if(len(opt_parser.reuse_train_emb_list) == 0):
    model.test(au_emb=au_emb)
else:
    model.test(au_emb=None)

画像どうしの翻訳

In [ ]:
fls = glob.glob1('examples', 'pred_fls_*.txt')
fls.sort()

for i in range(0,len(fls)):
    fl = np.loadtxt(os.path.join('examples', fls[i])).reshape((-1, 68,3))
    fl[:, :, 0:2] = -fl[:, :, 0:2]
    fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

    if (ADD_NAIVE_EYE):
        fl = util.add_naive_eye(fl)

    # additional smooth
    fl = fl.reshape((-1, 204))
    fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 15, 3, axis=0)
    fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
    fl = fl.reshape((-1, 68, 3))

    ''' STEP 6: Imag2image translation '''
    model = Image_translation_block(opt_parser, single_test=True)
    with torch.no_grad():
        model.single_test(jpg=img, fls=fl, filename=fls[i], prefix=opt_parser.jpg.split('.')[0])
        print('finish image2image gen')
    os.remove(os.path.join('examples', fls[i]))

可視化

In [48]:
from IPython.display import HTML
from base64 import b64encode

for ain in ains:
  OUTPUT_MP4_NAME = '{}_pred_fls_{}_audio_embed.mp4'.format(
    opt_parser.jpg.split('.')[0],
    ain.split('.')[0]
    )
  mp4 = open('examples/{}'.format(OUTPUT_MP4_NAME),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

  print('Display animation: examples/{}'.format(OUTPUT_MP4_NAME))
  display(HTML("""
  <video width=600 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

Display animation: examples/obama_pred_fls_japaneseAPI_audio_embed.mp4
